# 🚀 Get started to validate the setup

This Jupyter notebook is recommended for workshop/education only.

Prerequisites:

1. Set up your computing environment
2. Install the required library in your Python environment
3. Select the correct kernel (`azureml_py310_sdkv2`) for your Jupyter notebook


## 1. Azure Open AI Test

---


In [ ]:
%load_ext autoreload
%autoreload 2

from common import check_kernel
check_kernel()

In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv()

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

if not aoai_api_version:
    aoai_api_version = os.getenv("OPENAI_API_VERSION")
if not aoai_deployment_name:
    aoai_deployment_name = os.getenv("DEPLOYMENT_NAME")

    
try:
    client = AzureOpenAI(
        azure_endpoint = aoai_api_endpoint,
        api_key        = aoai_api_key,
        api_version    = aoai_api_version
    )
    deployment_name = aoai_deployment_name
    print("=== Initialized AzuureOpenAI client ===")
    print(f"AZURE_OPENAI_ENDPOINT={aoai_api_endpoint}")
    print(f"AZURE_OPENAI_API_VERSION={aoai_api_version}")
    print(f"AZURE_OPENAI_DEPLOYMENT_NAME={aoai_deployment_name}")   
except (ValueError, TypeError) as e:
    print(e)

In [ ]:
# Create your prompt
system_message = """
당신은 고객들이 정보를 찾을 수 있도록 도와주는 AI 어시스턴트입니다. 어시스턴트로서 
를 사용하여 질문에 간결하고 간결하게, 그리고 개성 있는 방식으로 답변하고 
마크다운을 사용하여 간단하고 간결하게 답변하고 적절한 이모티콘으로 개인적인 감각을 더할 수도 있습니다.

"그런데, "로 시작하는 재치 있는 농담을 답변 마지막에 추가하세요. 답변의 농담 부분에서는 고객의 이름을 언급하지 마세요. 
농담은 질문한 특정 질문과 관련이 있어야 합니다.
예를 들어 텐트에 대한 질문인 경우 농담은 텐트와 구체적으로 관련된 것이어야 합니다.

다음으로 주어지는 Context를 사용하여 보다 개인화된 응답을 제공하세요. 한문장마다 줄을 바꿔가면서 작성하고, 한국어로 답변 바랍니다:
"""

In [ ]:
# Create your prompt
system_message = """
당신은 고객들이 정보를 찾을 수 있도록 도와주는 AI 어시스턴트입니다. 어시스턴트로서 
를 사용하여 질문에 간결하고 간결하게, 그리고 개성 있는 방식으로 답변하고 
마크다운을 사용하여 간단하고 간결하게 답변하고 적절한 이모티콘으로 개인적인 감각을 더할 수도 있습니다.

"그런데, "로 시작하는 재치 있는 농담을 답변 마지막에 추가하세요. 답변의 농담 부분에서는 고객의 이름을 언급하지 마세요. 
농담은 질문한 특정 질문과 관련이 있어야 합니다.
예를 들어 텐트에 대한 질문인 경우 농담은 텐트와 구체적으로 관련된 것이어야 합니다.

다음으로 주어지는 Context를 사용하여 보다 개인화된 응답을 제공하세요. 한문장마다 줄을 바꿔가면서 작성하고, 한국어로 답변 바랍니다:
"""
context = """
    알파인 익스플로러 텐트는 탈착식 칸막이가 있어 프라이버시를 보장합니다, 
    통풍을 위한 수많은 메쉬 창과 조절 가능한 통풍구, 그리고 
    방수 설계가 특징입니다. 아웃도어 필수품을 보관할 수 있는 내장형 기어 로프트도 
    장비 로프트가 내장되어 있습니다. 요컨대, 프라이버시, 편안함, 
    편리함이 조화를 이루고 있어 자연 속 제2의 집과도 같은 숙소입니다!
"""
question = "알파인 익스플로러 텐트는 어떤 특징이 있나요?"

user_message = f"""
Context: {context}
Question: {question}
"""

# Simple API Call
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
  temperature=0.7,
  max_tokens=300
)

print(response.choices[0].message.content)

## (Optional) 2. Azure Document Inteligence Test

---


In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import ContentFormat

doc_intelligence_endpoint = os.getenv("AZURE_DOC_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOC_INTELLIGENCE_KEY")

try:
    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=doc_intelligence_endpoint, 
        credential=AzureKeyCredential(doc_intelligence_key),
        headers={"x-ms-useragent":"sample-code-figure-understanding/1.0.0"},
    )
    print("=== Initialized DocumentIntelligenceClient ===")
    print(f"AZURE_DOC_INTELLIGENCE_ENDPOINT={doc_intelligence_endpoint}")    
except (ValueError, TypeError) as e:
    print(e)
    
raw_data_dir = "../1_synthetic-qa-generation/raw_data"
file_path = f"{raw_data_dir}/pdf/en-imagenet-training-wrote-by-daekeun.pdf"

In [ ]:
with open(file_path, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", analyze_request=f, content_type="application/octet-stream", 
        output_content_format=ContentFormat.MARKDOWN 
    )

result = poller.result()
md_content = result.content
print(md_content)

## 3. Azure ML Test

---


In [ ]:
import os
import yaml
from datetime import datetime
snapshot_date = datetime.now().strftime("%Y-%m-%d")

with open('../2_slm-fine-tuning-mlstudio/phi3/config.yml') as f:
    d = yaml.load(f, Loader=yaml.FullLoader)
    
AZURE_SUBSCRIPTION_ID = d['config']['AZURE_SUBSCRIPTION_ID']
AZURE_RESOURCE_GROUP = d['config']['AZURE_RESOURCE_GROUP']
AZURE_WORKSPACE = d['config']['AZURE_WORKSPACE']
AZURE_DATA_NAME = d['config']['AZURE_DATA_NAME']    
DATA_DIR = d['config']['DATA_DIR']
CLOUD_DIR = d['config']['CLOUD_DIR']
HF_MODEL_NAME_OR_PATH = d['config']['HF_MODEL_NAME_OR_PATH']
IS_DEBUG = d['config']['IS_DEBUG']
USE_LOWPRIORITY_VM = d['config']['USE_LOWPRIORITY_VM']


print(f"AZURE_SUBSCRIPTION_ID={AZURE_SUBSCRIPTION_ID}")
print(f"AZURE_RESOURCE_GROUP={AZURE_RESOURCE_GROUP}")
print(f"AZURE_WORKSPACE={AZURE_WORKSPACE}")
print(f"AZURE_DATA_NAME={AZURE_DATA_NAME}")
print(f"DATA_DIR={DATA_DIR}")
print(f"CLOUD_DIR={CLOUD_DIR}")
print(f"HF_MODEL_NAME_OR_PATH={HF_MODEL_NAME_OR_PATH}")
print(f"IS_DEBUG={IS_DEBUG}")
print(f"USE_LOWPRIORITY_VM={USE_LOWPRIORITY_VM}")

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.core.exceptions import HttpResponseError

credential = DefaultAzureCredential()
ml_client = MLClient(credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE)

# from azure.identity import ClientSecretCredential
# credentials = ClientSecretCredential(
#     client_id=client_id,
#     client_secret=client_secret,
#     tenant_id=tenant_id
# )

try:
    workspace = ml_client.workspaces.get(name=AZURE_WORKSPACE)
    print(f"Connected to Azure ML Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")